In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import unicodedata
import re
from difflib import get_close_matches

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

# ========== Step 1: Setup ==========
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
date_str = yesterday.strftime('%Y%m%d')
scoreboard_url = f"https://www.espn.com/mlb/scoreboard/_/date/{date_str}"

options = Options()
options.add_argument("--headless")  # Remove this if you want to watch it work
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

driver.get(scoreboard_url)
time.sleep(5)  # let content load

# ========== Step 2: Find all "Box Score" links ==========
scoreboard_sections = driver.find_elements(By.CLASS_NAME, "Scoreboard")
clickable_links = []

for section in scoreboard_sections:
    try:
        driver.execute_script("arguments[0].scrollIntoView(true);", section)
        time.sleep(1)
        callout_div = section.find_element(By.CLASS_NAME, "Scoreboard__Callouts")
        links = callout_div.find_elements(By.TAG_NAME, "a")
        for link in links:
            href = link.get_attribute("href")
            if href and "/mlb/boxscore/_/gameId/" in href:
                clickable_links.append(link)
    except Exception as e:
        print("❌ Error collecting box score link:", e)

print(f"\n🎯 Found {len(clickable_links)} box score games for {date_str}\n")

# ========== Step 3: Helper to parse tables ==========
def parse_table(table):
    thead = table.find('thead')
    if thead:
        headers = [th.get_text(strip=True) for th in thead.find_all('th')]
    else:
        first_row = table.find('tbody').find('tr')
        headers = ['Col' + str(i+1) for i in range(len(first_row.find_all('td')))]
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        row = [td.get_text(strip=True) for td in tr.find_all('td')]
        row += [''] * (len(headers) - len(row))
        rows.append(row)
    return pd.DataFrame(rows, columns=headers)

# ========== Step 4: Click each, scrape, return ==========
all_data = []

for i in range(len(clickable_links)):
    try:
        # 🔁 REFIND all clickable boxscore links fresh
        scoreboard_sections = driver.find_elements(By.CLASS_NAME, "Scoreboard")
        fresh_links = []
        for section in scoreboard_sections:
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", section)
                time.sleep(0.5)
                callout_div = section.find_element(By.CLASS_NAME, "Scoreboard__Callouts")
                links = callout_div.find_elements(By.TAG_NAME, "a")
                for link in links:
                    href = link.get_attribute("href")
                    if href and "/mlb/boxscore/_/gameId/" in href:
                        fresh_links.append(link)
            except:
                continue

        # ✅ Now click the i-th fresh link
        print(f"🔍 Clicking into game {i+1} of {len(clickable_links)}...")
        driver.execute_script("arguments[0].click();", fresh_links[i])
        time.sleep(3)

        # Scrape data
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tables = soup.find_all('table', class_='Table')
        print(f"🧾 Found {len(tables)} tables")

        if len(tables) < 6:
            print("⚠️ Skipping — not enough tables\n")
            driver.back()
            time.sleep(2)
            continue

        away_names = parse_table(tables[2])
        away_stats = parse_table(tables[3])
        home_names = parse_table(tables[4])
        home_stats = parse_table(tables[5])

        away_df = pd.concat([away_names, away_stats], axis=1)
        home_df = pd.concat([home_names, home_stats], axis=1)

        max_len = max(len(home_df), len(away_df))
        home_df = home_df.reindex(range(max_len)).reset_index(drop=True)
        away_df = away_df.reindex(range(max_len)).reset_index(drop=True)

        away_df.columns = [f"away_{col}" for col in away_df.columns]
        home_df.columns = [f"home_{col}" for col in home_df.columns]

        merged = pd.concat([away_df, home_df], axis=1)
        all_data.append(merged)

        print("✅ Scraped successfully\n")
        driver.back()
        time.sleep(3)

    except Exception as e:
        print(f"❌ Error during scraping game {i+1}: {e}")
        try:
            driver.back()
            time.sleep(3)
        except:
            pass
        continue


# ========== Step 5: Combine all games ==========
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print("\n✅ FINAL PREVIEW:")
    print(final_df.head(20))
    # final_df.to_csv(f"boxscores_{date_str}.csv", index=False)  # optional save
else:
    print("⚠️ No data scraped.")


❌ Error collecting box score link: Message: no such element: Unable to locate element: {"method":"css selector","selector":".Scoreboard__Callouts"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF77C12CF25+75717]
	GetHandleVerifier [0x00007FF77C12CF80+75808]
	(No symbol) [0x00007FF77BEF8F9A]
	(No symbol) [0x00007FF77BF4F4C6]
	(No symbol) [0x00007FF77BF4F77C]
	(No symbol) [0x00007FF77BF41D7C]
	(No symbol) [0x00007FF77BF773BF]
	(No symbol) [0x00007FF77BF41C46]
	(No symbol) [0x00007FF77BF77590]
	(No symbol) [0x00007FF77BF9F39C]
	(No symbol) [0x00007FF77BF77153]
	(No symbol) [0x00007FF77BF40421]
	(No symbol) [0x00007FF77BF411B3]
	GetHandleVerifier [0x00007FF77C42D6FD+3223453]
	GetHandleVerifier [0x00007FF77C427CA2+3200322]
	GetHandleVerifier [0x00007FF77C445AD3+3322739]
	GetHandleVerifier [0x00007FF77C1469FA+180890

In [30]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# … your Step 1 setup …

driver.get(scoreboard_url)

# wait until at least one boxscore link appears
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'a[href*="/mlb/boxscore/_/gameId/"]')
    )
)

# ==== Step 2: collect all the hrefs once, to avoid stale elements ====
boxscore_elements = driver.find_elements(
    By.CSS_SELECTOR,
    'a[href*="/mlb/boxscore/_/gameId/"]'
)
boxscore_hrefs = [el.get_attribute('href') for el in boxscore_elements]
print(f"\n🎯 Found {len(boxscore_hrefs)} games for {date_str}\n")


# ==== Step 3: (your parse_table helper stays the same) ====


# ==== Step 4: loop through the hrefs, driver.get each one ====
all_data = []

for idx, href in enumerate(boxscore_hrefs, start=1):
    print(f"🔍 Loading game {idx} of {len(boxscore_hrefs)} → {href}")
    driver.get(href)
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, "table"))
    )
    soup   = BeautifulSoup(driver.page_source, "html.parser")
    tables = soup.find_all("table", class_="Table")
    print(f"🧾 Found {len(tables)} tables")

    if len(tables) < 6:
        print("⚠️ Skipping — not enough tables\n")
        continue

    # parse each quarter of the boxscore
    away_names = parse_table(tables[2])
    away_stats = parse_table(tables[3])
    home_names = parse_table(tables[4])
    home_stats = parse_table(tables[5])

    # combine name & stats side-by-side
    away_df = pd.concat([away_names, away_stats], axis=1)
    home_df = pd.concat([home_names, home_stats], axis=1)

    # pad shorter one so they align
    max_len = max(len(away_df), len(home_df))
    away_df = away_df.reindex(range(max_len)).reset_index(drop=True)
    home_df = home_df.reindex(range(max_len)).reset_index(drop=True)

    # prefix columns so we know which side is which
    away_df.columns = [f"away_{col}" for col in away_df.columns]
    home_df.columns = [f"home_{col}" for col in home_df.columns]

    # ← **THIS** is what you were missing before
    merged = pd.concat([away_df, home_df], axis=1)
    all_data.append(merged)

    print("✅ Scraped successfully\n")

# …then later…
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df.head())
else:
    print("⚠️ No data scraped.")


# … Step 5 combining remains unchanged …


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF77C12CF25+75717]
	GetHandleVerifier [0x00007FF77C12CF80+75808]
	(No symbol) [0x00007FF77BEF8F9A]
	(No symbol) [0x00007FF77BF4F4C6]
	(No symbol) [0x00007FF77BF4F77C]
	(No symbol) [0x00007FF77BFA2577]
	(No symbol) [0x00007FF77BF773BF]
	(No symbol) [0x00007FF77BF9F39C]
	(No symbol) [0x00007FF77BF77153]
	(No symbol) [0x00007FF77BF40421]
	(No symbol) [0x00007FF77BF411B3]
	GetHandleVerifier [0x00007FF77C42D6FD+3223453]
	GetHandleVerifier [0x00007FF77C427CA2+3200322]
	GetHandleVerifier [0x00007FF77C445AD3+3322739]
	GetHandleVerifier [0x00007FF77C1469FA+180890]
	GetHandleVerifier [0x00007FF77C14E0FF+211359]
	GetHandleVerifier [0x00007FF77C135274+109332]
	GetHandleVerifier [0x00007FF77C135422+109762]
	GetHandleVerifier [0x00007FF77C11BA39+4825]
	BaseThreadInitThunk [0x00007FFD0993E8D7+23]
	RtlUserThreadStart [0x00007FFD0BAFBF6C+44]


In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

# ========== Step 1: Setup ==========
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
date_str = yesterday.strftime('%Y%m%d')
scoreboard_url = f"https://www.espn.com/mlb/scoreboard/_/date/{date_str}"

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

# ========== Step 2: Collect boxscore URLs ==========
driver.get(scoreboard_url)
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="/mlb/boxscore/_/gameId/"]'))
)
time.sleep(3)
boxscore_elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/mlb/boxscore/_/gameId/"]')
boxscore_hrefs = [el.get_attribute('href') for el in boxscore_elements]
print(f"\n🎯 Found {len(boxscore_hrefs)} games for {date_str}\n")

# ========== Step 3: parse_table helper ==========
def parse_table(table):
    # 1) Look for a real <thead>
    thead = table.find('thead')
    if thead:
        headers = [th.get_text(strip=True) for th in thead.find_all('th')]
        rows_tr = table.select('tbody tr')
    else:
        # 2) No <thead> → inspect the first <tr> in the body
        trs = table.select('tbody tr')
        if not trs:
            return None
        first_cells = [td.get_text(strip=True) for td in trs[0].find_all('td')]
        # If most cells have letters, treat that row as the header
        alpha_count = sum(any(c.isalpha() for c in cell) for cell in first_cells)
        if alpha_count >= len(first_cells) / 2:
            headers = first_cells
            rows_tr = trs[1:]   # skip that header row
        else:
            headers = [f"Col{i+1}" for i in range(len(first_cells))]
            rows_tr = trs      # no header row to skip

    # 3) Build the data rows, padding or trimming as needed
    data = []
    for tr in rows_tr:
        cells = [td.get_text(strip=True) for td in tr.find_all('td')]
        cells = (cells + [''] * len(headers))[: len(headers)]
        data.append(cells)

    return pd.DataFrame(data, columns=headers)




# ========== Step 4: Loop through each game, scrape batting only ==========
all_data = []

for idx, href in enumerate(boxscore_hrefs, start=1):
    print(f"🔍 Loading game {idx} of {len(boxscore_hrefs)} → {href}")
    driver.get(href)

    # wait for the tab headers
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button"))
    )

    # click the “Batting” tab
    try:
        bat_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//button[normalize-space(text())='Batting']"
            ))
        )
        bat_tab.click()
        # wait for the batting sections to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.Boxscore__Category"))
        )
        time.sleep(0.5)
    except Exception as e:
        print("⚠️ Could not switch to Batting tab:", e)
        continue

    # parse the page
    soup = BeautifulSoup(driver.page_source, "html.parser")
    team_divs = soup.select("div.Boxscore__Category > div.Boxscore__Team")
    if len(team_divs) < 2:
        print("⚠️ Couldn't find away/home teams")
        continue

    # helper to scrape one side
    def scrape_div(div, side):
        fixed_tbl = div.select_one("div.ResponsiveTable--fixed-left table")
        stat_tbl  = div.select_one("div.Table__ScrollerWrapper table")
        names_df  = parse_table(fixed_tbl)
        stats_df  = parse_table(stat_tbl)
        df        = pd.concat([names_df, stats_df], axis=1)
        df.columns = [f"{side}_{c}" for c in df.columns]
        return df

    away_df = scrape_div(team_divs[0], "away")
    home_df = scrape_div(team_divs[1], "home")
    merged  = pd.concat([away_df, home_df], axis=1)

    all_data.append(merged)
    print("✅ Scraped batting stats successfully\n")

# ========== Step 5: Combine all games ==========
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df.head())
    print("\nColumns:")
    print(final_df.columns.tolist())
else:
    print("⚠️ No data scraped.")



TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF77C12CF25+75717]
	GetHandleVerifier [0x00007FF77C12CF80+75808]
	(No symbol) [0x00007FF77BEF8F9A]
	(No symbol) [0x00007FF77BF4F4C6]
	(No symbol) [0x00007FF77BF4F77C]
	(No symbol) [0x00007FF77BFA2577]
	(No symbol) [0x00007FF77BF773BF]
	(No symbol) [0x00007FF77BF9F39C]
	(No symbol) [0x00007FF77BF77153]
	(No symbol) [0x00007FF77BF40421]
	(No symbol) [0x00007FF77BF411B3]
	GetHandleVerifier [0x00007FF77C42D6FD+3223453]
	GetHandleVerifier [0x00007FF77C427CA2+3200322]
	GetHandleVerifier [0x00007FF77C445AD3+3322739]
	GetHandleVerifier [0x00007FF77C1469FA+180890]
	GetHandleVerifier [0x00007FF77C14E0FF+211359]
	GetHandleVerifier [0x00007FF77C135274+109332]
	GetHandleVerifier [0x00007FF77C135422+109762]
	GetHandleVerifier [0x00007FF77C11BA39+4825]
	BaseThreadInitThunk [0x00007FFD0993E8D7+23]
	RtlUserThreadStart [0x00007FFD0BAFBF6C+44]


In [32]:
import requests
import pandas as pd
import datetime

# ========== Step 1: get yesterday’s date ==========
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
date_str  = yesterday.strftime("%Y%m%d")

# ========== Step 2: fetch scoreboard JSON ==========
board_url = (
    "https://site.api.espn.com/apis/site/v2/"
    f"sports/baseball/mlb/scoreboard?dates={date_str}"
)
resp  = requests.get(board_url)
resp.raise_for_status()
board = resp.json()  # contains an “events” list :contentReference[oaicite:0]{index=0}

all_data = []

# ========== Step 3: for each game, fetch its summary JSON ==========
for ev in board.get("events", []):
    game_id = ev["id"]
    print(f"🔍 Loading game {game_id}")

    summ_url  = "https://site.api.espn.com/apis/site/v2/sports/baseball/mlb/summary"
    summ_resp = requests.get(summ_url, params={"event": game_id})
    if summ_resp.status_code != 200:
        print("⚠️ Summary fetch failed, skipping")
        continue

    summary = summ_resp.json()
    teams   = summary.get("boxscore", {}).get("teams", [])
    if len(teams) != 2:
        print("⚠️ Unexpected boxscore format, skipping")
        continue

    side_dfs = []

    # ========== Step 4: extract batting for away & home ==========
    for side_obj, prefix in zip(teams, ("away", "home")):
        stats_blocks = side_obj.get("statistics", [])
        batting = next((
            b for b in stats_blocks
            if b.get("name", "").lower() == "batting"
        ), None)

        if not batting:
            print(f"⚠️ No batting block for {prefix}, skipping game")
            side_dfs = []
            break

        # ESPN sometimes returns labels+values, or splits→stats
        if "labels" in batting and "values" in batting:
            labels = batting["labels"]
            values = batting["values"]
        else:
            # fallback to splits
            labels = [stat["name"] for stat in batting.get("splits", [])[0].get("stats", [])]
            values = [
                [st.get("value", st.get("displayValue", "")) for st in grp.get("stats", [])]
                for grp in batting.get("splits", [])
            ]

        df = pd.DataFrame(values, columns=labels)
        df = df.add_prefix(f"{prefix}_")
        side_dfs.append(df)

    if len(side_dfs) == 2:
        merged = pd.concat(side_dfs, axis=1)
        all_data.append(merged)
        print("✅ Scraped batting stats successfully\n")

# ========== Step 5: combine all games ==========
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df.head())
    print("\nColumns:")
    print(final_df.columns.tolist())
else:
    print("⚠️ No data collected.")


🔍 Loading game 401695466


IndexError: list index out of range

In [16]:
# final_df.to_csv('../../batter_record_1_hit/stats/batter_stats_2025.csv', mode='a', header=False, index=True)
# once final_df exists:
print(final_df.columns.tolist())


['away_hitters', 'away_AB', 'away_R', 'away_H', 'away_RBI', 'away_HR', 'away_BB', 'away_K', 'away_AVG', 'away_OBP', 'away_SLG', 'home_pitchers', 'home_IP', 'home_H', 'home_R', 'home_ER', 'home_BB', 'home_K', 'home_HR', 'home_PC-ST', 'home_ERA']


In [11]:
batter_team_2025 = pd.read_csv('../../batter_record_1_hit/stats/batters_team_2025.csv')

In [12]:
rbi_def_rank_2025=pd.read_csv('../def_scraper/batter_rbi_def_rank.csv')
hits_def_rank_2025=pd.read_csv('../def_scraper/batter_hits_def_rank.csv')
runs_def_rank_2025=pd.read_csv('../def_scraper/batter_runs_def_rank.csv')
# rbi_def_rank_2025=pd.read_csv('../def_scraper/batter_rbi_def_rank.csv')

In [13]:
final_df_2=final_df

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
final_df_2=final_df_2[['away_hitters','away_R','away_H','away_K','away_RBI','home_hitters','home_R','home_H','home_K','home_RBI']]
final_df_2

KeyError: "['home_hitters', 'home_RBI'] not in index"

In [ ]:
#normalizing the names, getting rid of accents
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [ ]:
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)


In [ ]:
# dropping other teams the same player played for, bc they got traded, 2TM stands for 2 teams etc....
rows_to_drop = set()

# Loop through the DataFrame
for i, val in batter_team_2025['team'].items():
    if val == '2TM':
        rows_to_drop.update(range(i, i+2))  # this row + 1 more
    elif val == '3TM':
        rows_to_drop.update(range(i, i+3))  # this row + 2 more
    elif val == '4TM':
        rows_to_drop.update(range(i, i+4))  # this row + 3 more
    elif val == '5TM':
        rows_to_drop.update(range(i, i+5))  # this row + 3 more

# Drop rows
batter_team_2025 = batter_team_2025.drop(rows_to_drop).reset_index(drop=True)

In [ ]:
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

In [ ]:
batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)

In [ ]:
batter_team_2025

In [ ]:
#changing names to match other files
stats_2025=final_df_2

In [ ]:
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # Normalize unicode to remove accents (e.g., Hernández → Hernandez)
    raw = unicodedata.normalize('NFKD', raw).encode('ascii', 'ignore').decode('utf-8')

    # Remove leading garbage (e.g., bT. Taylor → T. Taylor)
    raw = re.sub(r"^[^A-Z]*", "", raw)

    # Fix missing space after period (e.g., T.Taylor → T. Taylor)
    raw = re.sub(r"([A-Z])\.([A-Z])", r"\1. \2", raw)

    # Remove trailing position labels (e.g., PH-CF, 3B, DH)
    raw = re.sub(r"[A-Z]{1,3}(-[A-Z]{1,3})?$", "", raw.strip())

    # Remove anything else weird
    raw = re.sub(r"[^\w\s\.\'\-]", "", raw)

    # Match short name: first initial + last name (including compound last names)
    match = re.match(r"([A-Z]\.\s(?:[A-Z][a-z]+(?:\s|['’\-]))*[A-Z][a-z]+)", raw)
    return match.group(1).strip() if match else None


# Apply cleaner to get short names
stats_2025['short_name'] = stats_2025['away_hitters'].apply(clean_name)

# Match short names to full names
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # first initial match
            return name

    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Now apply matching
full_name_list = batter_team_2025['player'].tolist()
stats_2025['matched_name'] = stats_2025['short_name'].apply(lambda x: match_full_name(x, full_name_list))


In [ ]:
stats_2025

In [ ]:
stats_2025=stats_2025.rename(columns={"matched_name":"away_batters"})

In [ ]:
merged_df_5=stats_2025.drop(columns=['away_hitters','short_name'])

In [ ]:
merged_df_5

In [ ]:
def clean_name(raw):
    if not isinstance(raw, str):
        return None

    # Normalize unicode to remove accents (e.g., Hernández → Hernandez)
    raw = unicodedata.normalize('NFKD', raw).encode('ascii', 'ignore').decode('utf-8')

    # Remove leading garbage (e.g., bT. Taylor → T. Taylor)
    raw = re.sub(r"^[^A-Z]*", "", raw)

    # Fix missing space after period (e.g., T.Taylor → T. Taylor)
    raw = re.sub(r"([A-Z])\.([A-Z])", r"\1. \2", raw)

    # Remove trailing position labels (e.g., PH-CF, 3B, DH)
    raw = re.sub(r"[A-Z]{1,3}(-[A-Z]{1,3})?$", "", raw.strip())

    # Remove anything else weird
    raw = re.sub(r"[^\w\s\.\'\-]", "", raw)

    # Match short name: first initial + last name (including compound last names)
    match = re.match(r"([A-Z]\.\s(?:[A-Z][a-z]+(?:\s|['’\-]))*[A-Z][a-z]+)", raw)
    return match.group(1).strip() if match else None


# Apply cleaner to get short names
merged_df_5['short_name'] = merged_df_5['home_hitters'].apply(clean_name)

# Match short names to full names
def match_full_name(short_name, full_names):
    if not isinstance(short_name, str):
        return None

    last_name = short_name.split()[-1]
    full_names = [name for name in full_names if isinstance(name, str)]

    possible_matches = [name for name in full_names if last_name in name]
    for name in possible_matches:
        if name[0] == short_name[0]:  # first initial match
            return name

    close = get_close_matches(short_name, full_names, n=1, cutoff=0.5)
    return close[0] if close else None

# Now apply matching
full_name_list = batter_team_2025['player'].tolist()
merged_df_5['matched_name'] = merged_df_5['short_name'].apply(lambda x: match_full_name(x, full_name_list))


In [ ]:
merged_df_6=merged_df_5.rename(columns={"matched_name":"home_batters"})

In [ ]:
merged_df_6=merged_df_6.drop(columns=['home_hitters','short_name'])

In [ ]:
merged_df_6

In [ ]:
# Rename columns
# merged_df_7.rename(columns={
#     'player_x':'away_hitters',
#     'player_y':'home_hitters',
#     'team_x':'away_team',
#     'team_y':'home_team'
# }, inplace=True)

In [ ]:
away=merged_df_6[['away_batters','away_R','away_H','away_K','away_RBI']]
home=merged_df_6[['home_batters','home_R','home_H','home_K','home_RBI']]

In [ ]:
# Rename columns
away.rename(columns={
    'away_batters':'player',
    'away_K':'strikeouts',
    'away_R':'runs',
    'away_H':'hits',
    'away_RBI':'RBI'
}, inplace=True)

# Rename columns
home.rename(columns={
    'home_batters':'player',
    'home_K':'strikeouts',
    'home_R':'runs',
    'home_H':'hits',
    'home_RBI':'RBI'
}, inplace=True)



In [ ]:
stacked_df = pd.concat([away, home], ignore_index=True)


In [ ]:
stacked_df

In [ ]:
stacked_df.to_csv('mlb_results.csv')